# Portfolio Optimization

A portfolio manager oversees multiple assets $(i=1,\dots,N)$ and a bank account $(i=N+1)$. For a specified number of stages $T$, the manager wants to maximize his utility by dynamically reblanacing the portfolio.
Let $\{r_{it}\}$ be   the return process of   asset $i$. At the end of each period, the position of $i^{th}$ asset $x_{it}$ equals the start position $x_{i,t-1}$, plus the realized return $r_{it} x_{i,t-1}$ during the period, plus the newly long positions $b_{it}$, minus the newly short positions $s_{it}$. Transaction costs are $f_b,f_s$ for buying and selling respectively. The capital in the bank account will be adjusted accordingly.

We consider a simple asset pricing model that decomposes the excess return as the return explained by Capital Asset Pricing Model (CAPM), alpha and idiosyncratic risk,
\begin{equation*}
    r_{it}  =  r_{ft} + \beta_i (r_{Mt}-r_{ft}) + \epsilon_{it}, \textrm{where}~ \epsilon_{it}\overset{i.i.d}{\sim} N(\alpha_i,\sigma_i),
\end{equation*}
where $\alpha_i,\beta_i,\sigma_i$ are assumed to be constant. We refer to $\{r_{ft} + \beta_i (r_{Mt}-r_{ft})\}$ as the market-exposure return process and $\{\epsilon_{it}\}$ as the idiosyncratic return process. The market return process $\{r_{Mt}\}$ is modelled as a first-order autoregressive process (AR) with normal generalized autoregressive conditional heteroscedastic GARCH(1,1) innovation due to,
\begin{align*}
    & r_{Mt} = \mu + \phi r_{M,t-1} + \epsilon_{Mt},\\
    & \epsilon_{Mt} = \sigma_{Mt} e_{Mt},\\
    & \sigma_{Mt}^2 = \omega + \alpha_1 \epsilon_{M,t-1}^2 + \alpha_2 \sigma_{M,t-1}^2,\\
    & e_{Mt}\overset{i.i.d}{\sim} N(0,1).
\end{align*}

Formulation
----------------

\begin{align*}
    \max~& U(r_T^\top x_T)\\
    \textrm{s.t.}~&\forall t=1,\dots,T,\forall i=1,\dots,N,\\
    & x_{it} = x_{i,capm,t} + x_{i,idio,t} + x_{i,t-1} + b_{it} - s_{it},\\
    & x_{N+1,t} = (1 + r_{ft}) x_{N+1,t-1} - (1+f_{b}) \sum_{i=1}^N b_{it} + (1-f_{s}) \sum_{i=1}^N s_{it},\\
    & x_{i,capm,t} = \big[r_{ft} + \beta_{i} (r_{Mt}-r_{ft})\big] x_{i,t-1}\\
    & x_{i,idio,t} = \epsilon_{it} x_{i,t-1},\\
    & \epsilon_{it}\overset{i.i.d}{\sim} N(\alpha_i,\sigma_i), \{r_{Mt}\} \sim \textrm{AR}(1)-\textrm{GARCH}(1,1),\\
    & x_{i0} = 0, x_{N+1,0} = \$ 100 \\
    & b_{it},s_{it},x_{it},x_{N+1,t}\geq 0.
\end{align*}

Deterministic solution
----------------

In [1]:
using CSV, DataFrames
using JuMP, Gurobi
using Plots

rf = 0.0005
fee = 0.001

0.001

In [21]:
params = DataFrame(CSV.File("./data/parameters.csv"))
mu, phi, omega, alpha_1, alpha_2 = params[:,2]

coeffs = DataFrame(CSV.File("./data/coefficients.csv"))
alpha = coeffs[:,2]
beta = coeffs[:,3]
epsilon = coeffs[:,4];

prices = DataFrame(CSV.File("./data/prices.csv"))
prices = prices[:,2:end];

In [50]:
returns = similar(prices, 0)
for i in 1:size(prices)[2]-1
    m = collect(prices[i+1,:])./collect(prices[i,:])
    push!(returns, m)
end

In [52]:
T = 3
N = 5 # The first N assets in prices

m = Model(Gurobi.Optimizer)



Academic license - for non-commercial use only - expires 2022-01-02


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi